In [3]:
import pandas as pd
import tiktoken

In [4]:
sample_sub = pd.read_csv('/data/Project/NLP/detection_toxique/data/sample_submission.csv')

In [96]:
print(sample_sub.head(10))

                 id  toxic  severe_toxic  obscene  threat  insult  \
0  00001cee341fdb12    0.5           0.5      0.5     0.5     0.5   
1  0000247867823ef7    0.5           0.5      0.5     0.5     0.5   
2  00013b17ad220c46    0.5           0.5      0.5     0.5     0.5   
3  00017563c3f7919a    0.5           0.5      0.5     0.5     0.5   
4  00017695ad8997eb    0.5           0.5      0.5     0.5     0.5   
5  0001ea8717f6de06    0.5           0.5      0.5     0.5     0.5   
6  00024115d4cbde0f    0.5           0.5      0.5     0.5     0.5   
7  000247e83dcc1211    0.5           0.5      0.5     0.5     0.5   
8  00025358d4737918    0.5           0.5      0.5     0.5     0.5   
9  00026d1092fe71cc    0.5           0.5      0.5     0.5     0.5   

   identity_hate  
0            0.5  
1            0.5  
2            0.5  
3            0.5  
4            0.5  
5            0.5  
6            0.5  
7            0.5  
8            0.5  
9            0.5  


In [5]:
data_train = pd.read_csv('/data/Project/NLP/detection_toxique/data/train.csv')

In [3]:
print(data_train.head(10))

                 id                                       comment_text  toxic  \
0  0000997932d777bf  Explanation\nWhy the edits made under my usern...      0   
1  000103f0d9cfb60f  D'aww! He matches this background colour I'm s...      0   
2  000113f07ec002fd  Hey man, I'm really not trying to edit war. It...      0   
3  0001b41b1c6bb37e  "\nMore\nI can't make any real suggestions on ...      0   
4  0001d958c54c6e35  You, sir, are my hero. Any chance you remember...      0   
5  00025465d4725e87  "\n\nCongratulations from me as well, use the ...      0   
6  0002bcb3da6cb337       COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK      1   
7  00031b1e95af7921  Your vandalism to the Matt Shirvington article...      0   
8  00037261f536c51d  Sorry if the word 'nonsense' was offensive to ...      0   
9  00040093b2687caa  alignment on this subject and which are contra...      0   

   severe_toxic  obscene  threat  insult  identity_hate  
0             0        0       0       0          

In [7]:
data_test = pd.read_csv('/data/Project/NLP/detection_toxique/data/test.csv')
print(data_test.head(10))

                 id                                       comment_text
0  00001cee341fdb12  Yo bitch Ja Rule is more succesful then you'll...
1  0000247867823ef7  == From RfC == \n\n The title is fine as it is...
2  00013b17ad220c46  " \n\n == Sources == \n\n * Zawe Ashton on Lap...
3  00017563c3f7919a  :If you have a look back at the source, the in...
4  00017695ad8997eb          I don't anonymously edit articles at all.
5  0001ea8717f6de06  Thank you for understanding. I think very high...
6  00024115d4cbde0f  Please do not add nonsense to Wikipedia. Such ...
7  000247e83dcc1211                   :Dear god this site is horrible.
8  00025358d4737918  " \n Only a fool can believe in such numbers. ...
9  00026d1092fe71cc  == Double Redirects == \n\n When fixing double...


In [8]:
train_tweets = data_train.comment_text
test_tweets = data_test.comment_text
print(train_tweets.head(10))
print(f"Number of training samples: {len(train_tweets)}")
print(f"Number of test samples: {len(test_tweets)}")

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
5    "\n\nCongratulations from me as well, use the ...
6         COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
7    Your vandalism to the Matt Shirvington article...
8    Sorry if the word 'nonsense' was offensive to ...
9    alignment on this subject and which are contra...
Name: comment_text, dtype: object
Number of training samples: 159571
Number of test samples: 153164


In [93]:
label_columns = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
# 将标签转换为 list
labels_list = data_train[label_columns].values.tolist()

# 打印结果
print(labels_list[:10])

[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]


In [6]:
def find_longest_length(tweets):
    """
    Find the longest tweet in the entire training set.
    
    :param tweets: A pandas data series.
    
    Returns:
        max_length: Longest tweet length.
    """
    max_length = 0
    for i, text in enumerate(tweets):
        corpus = [
            word for word in text.split()
        ]
        if len(corpus) > max_length:
            max_length = len(corpus)
    return max_length

In [7]:
longest_sentence_length = find_longest_length(train_tweets)

print(f"Longest tweet: {longest_sentence_length} words")

Longest tweet: 1411 words


In [8]:
def find_avg_sentence_length(tweets):
    """
    Find the average sentence length among all
    the tweets.
    
    :param tweets: A pandas data series.
    
    Returns:
        Average sentence length.
    """
    sentence_lengths = []
    for i, text in enumerate(tweets):
        corpus = [
                word for word in text.split()
            ]
        sentence_lengths.append(len(corpus))
    return sum(sentence_lengths)/len(sentence_lengths)

In [9]:
average_length = find_avg_sentence_length(train_tweets)
print(f"Average sentence length: {average_length} words")  

Average sentence length: 67.27352714465661 words


In [10]:
# 加载分词器
tokenizer = tiktoken.get_encoding("cl100k_base")

In [85]:
# 计算每条推文的 token 数
tweet_lengths = train_tweets.apply(lambda x: len(tokenizer.encode(x)))

average_len = tweet_lengths.mean()
print(f"Average token length: {average_len}")
# 获取最大长度
max_len = tweet_lengths.max()
print(f"Maximum token length: {max_len}")
max_len = int(tweet_lengths.quantile(0.95))  # 使用 95% 分位数
print(f"95% quantile token length: {max_len}")

Average token length: 91.07166089076335
Maximum token length: 2908
95% quantile token length: 300


In [13]:
# 打印词汇表大小
vocab_size = tokenizer.n_vocab
print(f"Vocabulary size: {vocab_size}")

Vocabulary size: 100277


In [14]:
all_tokens = [token for tweet in train_tweets for token in tokenizer.encode(tweet)]
unique_tokens = set(all_tokens)

# 实际 vocab size
actual_vocab_size = len(unique_tokens)
print(f"Actual vocab size: {actual_vocab_size}")

Actual vocab size: 66435


In [12]:
MAX_LEN = int(max_len)
# Use these many top words from the dataset. If -1, use all words.
NUM_WORDS = vocab_size # Vocabulary size.
# Batch size.
BATCH_SIZE = 512
VALID_SPLIT = 0.20

In [15]:
train_list = train_tweets.tolist()

In [16]:
len(train_list)

159571

In [24]:
tokens_list = []
for i, text in enumerate(train_list):
    tokens = text.encode("utf-8") # raw bytes
    tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
    tokens_list.append(tokens)

In [27]:
def get_stats(tokens_list):
    counts = {}
    for i, ids in enumerate(tokens_list):
        for  pair in zip(ids, ids[1:]):
            counts[pair] = counts.get(pair, 0) + 1
    return counts

In [28]:
stats = get_stats(tokens_list)
print(stats)

{(69, 120): 3045, (120, 112): 20686, (112, 108): 111305, (108, 97): 147626, (97, 110): 674076, (110, 97): 107366, (97, 116): 539064, (116, 105): 487516, (105, 111): 261427, (111, 110): 646748, (110, 10): 3382, (10, 87): 9558, (87, 104): 22351, (104, 121): 26380, (121, 32): 508085, (32, 116): 1355607, (116, 104): 1093119, (104, 101): 849096, (101, 32): 1776386, (32, 101): 214069, (101, 100): 488733, (100, 105): 269430, (105, 116): 458391, (116, 115): 103557, (115, 32): 1041882, (32, 109): 318351, (109, 97): 179461, (97, 100): 139178, (100, 101): 239631, (32, 117): 128277, (117, 110): 112681, (110, 100): 393011, (101, 114): 653297, (114, 32): 488508, (109, 121): 46226, (117, 115): 214542, (115, 101): 360472, (114, 110): 42514, (97, 109): 104490, (109, 101): 289724, (32, 72): 47313, (72, 97): 10446, (97, 114): 405235, (114, 100): 59344, (100, 99): 911, (99, 111): 243952, (111, 114): 453540, (114, 101): 681396, (32, 77): 58520, (77, 101): 7988, (101, 116): 153498, (116, 97): 201474, (97, 1

In [29]:
print(sorted(((v,k) for k,v in stats.items()), reverse=True))

[(1776386, (101, 32)), (1355607, (32, 116)), (1114307, (32, 97)), (1093119, (116, 104)), (1041882, (115, 32)), (1012952, (116, 32)), (849096, (104, 101)), (822408, (105, 110)), (748752, (100, 32)), (681396, (114, 101)), (676121, (110, 32)), (674076, (97, 110)), (653297, (101, 114)), (646748, (111, 110)), (639861, (32, 105)), (572799, (111, 117)), (539064, (97, 116)), (536987, (32, 111)), (523599, (32, 115)), (508085, (121, 32)), (501384, (105, 115)), (492226, (32, 119)), (488733, (101, 100)), (488508, (114, 32)), (487516, (116, 105)), (478860, (111, 32)), (462139, (44, 32)), (458391, (105, 116)), (453540, (111, 114)), (439657, (104, 97)), (427295, (46, 32)), (422262, (116, 101)), (415268, (101, 115)), (413505, (116, 111)), (410569, (101, 110)), (405235, (97, 114)), (393011, (110, 100)), (392477, (108, 101)), (392004, (110, 103)), (389188, (32, 98)), (385551, (32, 99)), (360472, (115, 101)), (359839, (115, 116)), (358983, (97, 108)), (345555, (32, 112)), (326146, (118, 101)), (324590, (

In [30]:
def merge(tokens_list, pair, idx):
  newlists = []
  for x, ids in enumerate(tokens_list):
    newids = []
    i = 0
    while i < len(ids):
      if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
        newids.append(idx)
        i += 2
      else:
        newids.append(ids[i])
        i += 1
    newlists.append(newids)
  return newlists

In [97]:
def build_vocab(tokens_list, vocab_size=276):
    num_merges = vocab_size - 256
    merges = {} # (int, int) -> int
    for i in range(num_merges):
        stats = get_stats(tokens_list)
        pair = max(stats, key=stats.get)
        idx = 256 + i
        print(f"merging {pair} into a new token {idx}")
        tokens_list = merge(tokens_list, pair, idx)
        merges[pair] = idx
    return merges

In [40]:
merges = build_vocab(tokens_list, stats)

merging (101, 32) into a new token 256
merging (32, 116) into a new token 257
merging (115, 32) into a new token 258
merging (116, 32) into a new token 259
merging (105, 110) into a new token 260
merging (257, 104) into a new token 261
merging (97, 110) into a new token 262
merging (101, 114) into a new token 263
merging (111, 110) into a new token 264
merging (100, 32) into a new token 265
merging (111, 117) into a new token 266
merging (111, 114) into a new token 267
merging (121, 32) into a new token 268
merging (46, 32) into a new token 269
merging (111, 32) into a new token 270
merging (44, 32) into a new token 271
merging (101, 110) into a new token 272
merging (97, 114) into a new token 273
merging (261, 256) into a new token 274
merging (116, 105) into a new token 275


In [65]:
merges

{(101, 32): 256,
 (32, 116): 257,
 (115, 32): 258,
 (116, 32): 259,
 (105, 110): 260,
 (257, 104): 261,
 (97, 110): 262,
 (101, 114): 263,
 (111, 110): 264,
 (100, 32): 265,
 (111, 117): 266,
 (111, 114): 267,
 (121, 32): 268,
 (46, 32): 269,
 (111, 32): 270,
 (44, 32): 271,
 (101, 110): 272,
 (97, 114): 273,
 (261, 256): 274,
 (116, 105): 275}

In [42]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

In [54]:
def merge_tokens(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

In [71]:
def get_stats_tokens(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

In [76]:
def encode(tweets, merges):
    tokens_list = []
    for i, text in enumerate(tweets):
        tokens = list(text.encode("utf-8"))
        while len(tokens) >= 2:
            stats = get_stats_tokens(tokens)
            pair = min(stats, key=lambda p: merges.get(p, float("inf")))
            if pair not in merges:
                break # nothing else can be merged
            idx = merges[pair]
            tokens = merge_tokens(tokens, pair, idx)
        tokens_list.append(tokens)
    return tokens_list

In [72]:
def encode1(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_stats_tokens(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge_tokens(tokens, pair, idx)
  return tokens

print(encode1("Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"))

[69, 120, 112, 108, 262, 97, 275, 264, 10, 87, 104, 121, 274, 101, 100, 105, 116, 258, 109, 97, 100, 256, 117, 110, 100, 263, 32, 109, 268, 117, 115, 263, 110, 97, 109, 256, 72, 273, 100, 99, 267, 256, 77, 101, 116, 97, 108, 108, 105, 99, 97, 32, 70, 262, 32, 119, 263, 256, 114, 101, 118, 263, 116, 101, 100, 63, 32, 84, 104, 101, 268, 119, 263, 272, 39, 259, 118, 262, 100, 97, 108, 105, 115, 109, 115, 271, 106, 117, 115, 259, 99, 108, 111, 115, 117, 114, 256, 264, 32, 115, 111, 109, 256, 71, 65, 258, 97, 102, 116, 263, 32, 73, 32, 118, 111, 116, 101, 265, 97, 259, 78, 101, 119, 32, 89, 267, 107, 32, 68, 111, 108, 108, 258, 70, 65, 67, 269, 65, 110, 265, 112, 108, 101, 97, 115, 256, 100, 264, 39, 259, 114, 101, 109, 111, 118, 256, 116, 104, 256, 116, 101, 109, 112, 108, 97, 116, 256, 102, 114, 111, 109, 274, 116, 97, 108, 107, 32, 112, 97, 103, 256, 115, 260, 99, 256, 73, 39, 109, 32, 114, 101, 275, 114, 101, 265, 110, 111, 119, 46, 56, 57, 46, 50, 48, 53, 46, 51, 56, 46, 50, 55]


In [73]:
top10 = train_list[:10]

In [67]:
top10

["Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"]

In [94]:
input_tokens = encode(top10, merges)
print(input_tokens)

[[69, 120, 112, 108, 262, 97, 275, 264, 10, 87, 104, 121, 274, 101, 100, 105, 116, 258, 109, 97, 100, 256, 117, 110, 100, 263, 32, 109, 268, 117, 115, 263, 110, 97, 109, 256, 72, 273, 100, 99, 267, 256, 77, 101, 116, 97, 108, 108, 105, 99, 97, 32, 70, 262, 32, 119, 263, 256, 114, 101, 118, 263, 116, 101, 100, 63, 32, 84, 104, 101, 268, 119, 263, 272, 39, 259, 118, 262, 100, 97, 108, 105, 115, 109, 115, 271, 106, 117, 115, 259, 99, 108, 111, 115, 117, 114, 256, 264, 32, 115, 111, 109, 256, 71, 65, 258, 97, 102, 116, 263, 32, 73, 32, 118, 111, 116, 101, 265, 97, 259, 78, 101, 119, 32, 89, 267, 107, 32, 68, 111, 108, 108, 258, 70, 65, 67, 269, 65, 110, 265, 112, 108, 101, 97, 115, 256, 100, 264, 39, 259, 114, 101, 109, 111, 118, 256, 116, 104, 256, 116, 101, 109, 112, 108, 97, 116, 256, 102, 114, 111, 109, 274, 116, 97, 108, 107, 32, 112, 97, 103, 256, 115, 260, 99, 256, 73, 39, 109, 32, 114, 101, 275, 114, 101, 265, 110, 111, 119, 46, 56, 57, 46, 50, 48, 53, 46, 51, 56, 46, 50, 55], [68,

In [9]:
import regex as re
pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""")

print(re.findall(pat, train_tweets[0]))

['Explanation', '\n', 'Why', ' the', ' edits', ' made', ' under', ' my', ' username', ' Hardcore', ' Metallica', ' Fan', ' were', ' reverted', '?', ' They', ' weren', "'t", ' vandalisms', ',', ' just', ' closure', ' on', ' some', ' GAs', ' after', ' I', ' voted', ' at', ' New', ' York', ' Dolls', ' FAC', '.', ' And', ' please', ' don', "'t", ' remove', ' the', ' template', ' from', ' the', ' talk', ' page', ' since', ' I', "'m", ' retired', ' now', '.', '89', '.', '205', '.', '38', '.', '27']
